In [1]:
import pandas as pd
import utils
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV

In [2]:
from time import time
import numpy as np
from sklearn import svm
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc

In [3]:
from sklearn.externals import joblib
scale = joblib.load("models/scaler.sav")

### Feature Selection
Lasso 

In [4]:
train = pd.read_csv("data/final_data/clean_train.csv")
X,y,c_names = utils.setup_data(train,scale,'Team2_win',names=True)
train.head()

,Team1_first_downs,Team1_passing_first_downs,Team1_rushing_first_downs,Team1_rushing_yds,Team1_rushing_attempts,Team1_passing_attempts,Team1_passing_completions,Team1_passing_interceptions,Team1_avg_per_pass,Team1_pass_yds,...,Team2_third_down_suc,Team2_fourth_down_attempts,Team2_fourth_down_suc,Team1_third_down_conv_pct,Team2_third_down_conv_pct,Team1_fourth_down_conv_pct,Team2_fourth_down_conv_pct,Team1_turnover_margin,Team2_turnover_margin,Team2_win
0,22,14,7,291,48,20,14,0,7.6,152,...,5,3,0,0.428571,0.384615,1.0,0.0,-2,2,0
1,18,3,14,50,34,40,26,1,7.7,307,...,7,1,0,0.352941,0.388889,0.0,0.0,0,0,0
2,29,12,16,164,41,41,27,0,8.1,333,...,4,2,2,0.466667,0.285714,0.0,1.0,-1,1,0
3,25,13,8,234,43,31,14,2,4.7,146,...,5,2,1,0.333333,0.416667,1.0,0.5,2,-2,1
4,14,7,6,201,34,19,10,0,9.8,186,...,8,2,0,0.272727,0.444444,0.0,0.0,-4,4,0


In [5]:
test = pd.read_csv("data/final_data/clean_test.csv")
X_test,y_test = utils.setup_data(test,scale,target_col='Team2_win')
test.head()

,Team1_first_downs,Team1_passing_first_downs,Team1_rushing_first_downs,Team1_rushing_yds,Team1_rushing_attempts,Team1_passing_attempts,Team1_passing_completions,Team1_passing_interceptions,Team1_avg_per_pass,Team1_pass_yds,...,Team2_third_down_suc,Team2_fourth_down_attempts,Team2_fourth_down_suc,Team1_third_down_conv_pct,Team2_third_down_conv_pct,Team1_fourth_down_conv_pct,Team2_fourth_down_conv_pct,Team1_turnover_margin,Team2_turnover_margin,Team2_win
0,29,23,5,453,69,22,13,1,4.4,97,...,5,3,1,0.590909,0.357143,0.666667,0.333333,-2,2,0
1,27,17,5,317,44,45,24,1,2.8,124,...,8,0,0,0.315789,0.666667,1.000000,0.000000,-1,1,1
2,8,4,4,136,27,22,9,0,4.3,95,...,7,2,1,0.166667,0.368421,0.000000,0.500000,1,-1,1
3,21,12,8,293,56,24,13,1,7.0,169,...,6,0,0,0.200000,0.352941,0.666667,0.000000,1,-1,1
4,18,4,12,90,27,51,26,0,5.4,273,...,5,1,1,0.277778,0.333333,0.000000,1.000000,0,0,1


In [6]:
clf = LassoCV(cv=5)

# Set a minimum threshold of 0.25
sfm = SelectFromModel(clf, threshold='mean')
sfm.fit(X, y)
selectedFeatures = sfm.transform(X)

In [7]:
test_selected = sfm.transform(X_test)

### Modelling

In [8]:
svm_clf = svm.SVC(cache_size=7000)
params = {
    'kernel': ['linear','rbf','sigmoid'],
    'C' : np.arange(0.0, 0.5, 0.001),
    'tol' : np.arange(0.0, 0.1, 0.0001),
}

In [9]:
random_serach = RandomizedSearchCV(svm_clf,params,cv=10,random_state=10,refit = True)

In [10]:
start = time()
random_serach.fit(X,y)
print("RandomizedSearchCV took %.2f seconds." % ((time() - start)))

RandomizedSearchCV took 14.36 seconds.


In [11]:
utils.report(random_serach.cv_results_,n_top=10)

Model with rank: 1
Mean validation score: 0.947 (std: 0.017)
Parameters: {'tol': 0.0961, 'kernel': 'linear', 'C': 0.442}

Model with rank: 2
Mean validation score: 0.943 (std: 0.019)
Parameters: {'tol': 0.033600000000000005, 'kernel': 'linear', 'C': 0.268}

Model with rank: 3
Mean validation score: 0.942 (std: 0.019)
Parameters: {'tol': 0.0506, 'kernel': 'linear', 'C': 0.32}

Model with rank: 4
Mean validation score: 0.927 (std: 0.020)
Parameters: {'tol': 0.08410000000000001, 'kernel': 'sigmoid', 'C': 0.20500000000000002}

Model with rank: 4
Mean validation score: 0.927 (std: 0.019)
Parameters: {'tol': 0.0171, 'kernel': 'sigmoid', 'C': 0.384}

Model with rank: 6
Mean validation score: 0.927 (std: 0.018)
Parameters: {'tol': 0.0021000000000000003, 'kernel': 'sigmoid', 'C': 0.385}

Model with rank: 7
Mean validation score: 0.924 (std: 0.023)
Parameters: {'tol': 0.09290000000000001, 'kernel': 'rbf', 'C': 0.464}

Model with rank: 8
Mean validation score: 0.921 (std: 0.022)
Parameters: {'tol

In [12]:
SVC_final=svm.SVC(**random_serach.best_params_,probability=True)
SVC_final.fit(X,y)

SVC(C=0.442, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.0961, verbose=False)

In [13]:
import pickle
pickle.dump(SVC_final,open("models/SVC_model.sav",'wb'))

In [14]:
y_pred = SVC_final.predict(X_test)
y_pred_prob = SVC_final.predict_proba(X_test)[:, 1]

In [15]:
print("SVM Acc Score: {0:.3%}".format(accuracy_score(y_test,y_pred)))

SVM Acc Score: 92.232%
